# Neural Network implementation (using only numpy)

## 1. Implement Activation Functions
> sigmoid, tanh, relu, leaky_relu

## 2. Design a class Neural Network
> constructor, addHiddenLayer, parameter initializer, forward propagation, backward propagation, predict

## 3. Implement Constructor

## 4. Implement ```addHiddenLayer```, ```initialize_parameters```,  ```compute_cost```

## 5. Implement ```forward_propagation```, ```backward_propagation```,  ```update_parameters```

## 6. Implement ```run_model```, ```predict```

In [0]:
# Pakage Imports
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model

%matplotlib inline

In [0]:
#Activation Functions
def sigmoid(z, derivative = False):
    """Sigmoid Function"""
    
    if derivative:
      a = sigmoid(z)
      return a * (1 - a)
    
    return 1 / (1 + np.exp(-z))

def tanh(z, derivative = False):
    """Hyperbolic Tangent Function"""
    
    if derivative:
      a = tanh(z)
      return 1 - np.power(a, 2)
    
    return np.tanh(z)

def relu(z, derivative = False):
    """Rectified Linear Unit Function"""
    
    if derivative:
        return z > 0
    
    return np.maximum(0, z)

def leaky_relu(z, scale = 0.01, derivative = False):
    """Leaky ReLU"""
    
    if derivative:
        return z > 0 + (z <= 0) * scale
      
    return np.maximum(scale * z, z)

class NeuralNetwork:    
    def __init__(self, activation_function = sigmoid):
        self.n = [-1, 1]
        self.actFunc = [0, activation_function]
    
    def addHiddenLayer(self, number_of_units:int, activation_function = relu):
        self.n.insert(len(self.n) - 1, number_of_units)
        self.actFunc.insert(len(self.actFunc) - 1, activation_function)

    def initialize_parameters(self, X, scale = 0.01):
        number_of_layers = len(self.n) - 1
        
        print('# of Layers =', number_of_layers)
        
        self.W = [0] * (number_of_layers + 1)
        self.dW = [0] * (number_of_layers + 1)
        self.b = [0] * (number_of_layers + 1)
        self.db = [0] * (number_of_layers + 1)
        self.A = [0] * (number_of_layers + 1)
        self.Z = [0] * (number_of_layers + 1)
        self.dZ = [0] * (number_of_layers + 1)
        
        self.A[0] = X
        self.n[0] = X.shape[0]
        
        for i in range(1, len(self.n)):
            self.W[i] = np.random.randn(self.n[i], self.n[i - 1]) * scale
            self.b[i] = np.zeros((self.n[i], 1))
            
    def compute_cost(self, Y):
        Y_hat = self.A[-1]
        m = Y.shape[1] # number of example

        # Compute the cross-entropy cost
        logprobs = Y * np.log(Y_hat) + (1 - Y) * np.log(1 - Y_hat)
        cost = - np.sum(logprobs) / m

        cost = np.squeeze(cost)
        assert(isinstance(cost, float))

        return cost
      
    def forward_propagation(self):
        for i in range(1, len(self.n)):
            self.Z[i] = self.W[i] @ self.A[i - 1] + self.b[i]
            self.A[i] = self.actFunc[i](self.Z[i])
            
    def backward_propagation(self, Y):
        """
          Implement the backward propagation using the instructions above.

          Arguments:
          X -- input data of shape (2, number of examples)
          Y -- "true" labels vector of shape (1, number of examples)
        """
        
        m = self.A[0].shape[1]

        # Backward propagation: calculate dW, db
        
        self.dZ[-1] = self.A[-1] - Y
        self.dW[-1] = 1 / m * self.dZ[-1] @ self.A[-2].T
        self.db[-1] = 1 / m * np.sum(self.dZ[-1], axis = 1, keepdims = True)
        
        for i in range(len(self.A) - 2, 0, -1):
            self.dZ[i] = self.W[i + 1].T @ self.dZ[i + 1] * self.actFunc[i](self.A[i], derivative = True)
            self.dW[i] = 1 / m * self.dZ[i] @ self.A[i-1].T
            self.db[i] = 1 / m * np.sum(self.dZ[i], axis = 1, keepdims = True)

    def update_parameters(self, learning_rate):
        """
          Updates parameters using the gradient descent update rule
        """
        
        # Update rule for each parameter
        number_of_layers = len(self.n) - 1
        for i in range(1, number_of_layers + 1):
            self.W[i] = self.W[i] - learning_rate * self.dW[i]
            self.b[i] = self.b[i] - learning_rate * self.db[i]
    
    def run_model(self, X, Y, learning_rate = 0.01, num_iterations = 10000, print_cost = False):
        """
            Arguments:
            X -- dataset of shape (2, number of examples)
            Y -- labels of shape (1, number of examples)
            learning_rate -- Learning Rate of Gradient Descent
            num_iterations -- Number of iterations in gradient descent loop
            print_cost -- if True, print the cost every 1000 iterations
        """
        
        # Initialize parameters.
        self.initialize_parameters(X)        

        # Loop (gradient descent)
        for i in range(num_iterations):

            # Forward propagation.
            self.forward_propagation()

            # Cost function.
            cost = self.compute_cost(Y)

            # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
            self.backward_propagation(Y)

            # Gradient descent parameter update.
            self.update_parameters(learning_rate)

            # Print the cost every 1000 iterations
            if print_cost and i % 1000 == 0:
                print ("Cost after iteration %i: %f" %(i, cost))
                
    def predict(self, X, Y, threshold = 0.5):
        """
        Using the learned parameters, predicts a class for each example in X

        Arguments:
        X -- input data of size (n_x, m)
        Y -- output data of size (1, m)

        Returns
        predictions -- vector of predictions of our model
        """
        
        A = [0] * len(self.n)
        Z = [0] * len(self.n)
        A[0] = X

        # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
        for i in range(1, len(self.n)):
            Z[i] = self.W[i] @ A[i - 1] + self.b[i]
            A[i] = self.actFunc[i](Z[i])
            
        predictions = A[-1] > threshold

        return predictions

# Make a new neural network with two hidden layers

> ## First hidden layer has 5 hidden units.

> ## Second hidden layer has 4 hidden units.

In [0]:
network = NeuralNetwork()
network.addHiddenLayer(5)
network.addHiddenLayer(3)

# Make a training set and a test set with random numbers.

In [0]:
training_set_size, test_set_size = 10000, 1000
X = np.random.randn(5, training_set_size); Y = np.sum(X, axis = 0, keepdims = True) > 0; Y = Y.reshape(1, -1).astype('uint32')
test_X = np.random.randn(5, test_set_size); test_Y = np.sum(test_X, axis = 0, keepdims = True) > 0; test_Y = test_Y.reshape(1, -1).astype('uint32')

# Run the model.

In [14]:
network.run_model(X, Y, learning_rate = 0.03, num_iterations = 30000, print_cost = True)

# of Layers = 3
Cost after iteration 0: 0.693147
Cost after iteration 1000: 0.693142
Cost after iteration 2000: 0.693142
Cost after iteration 3000: 0.693142
Cost after iteration 4000: 0.693141
Cost after iteration 5000: 0.693140
Cost after iteration 6000: 0.693138
Cost after iteration 7000: 0.693133
Cost after iteration 8000: 0.693118
Cost after iteration 9000: 0.693047
Cost after iteration 10000: 0.692004
Cost after iteration 11000: 0.255167
Cost after iteration 12000: 0.076976
Cost after iteration 13000: 0.047438
Cost after iteration 14000: 0.035260
Cost after iteration 15000: 0.028509
Cost after iteration 16000: 0.024160
Cost after iteration 17000: 0.021111
Cost after iteration 18000: 0.018845
Cost after iteration 19000: 0.017084
Cost after iteration 20000: 0.015672
Cost after iteration 21000: 0.014512
Cost after iteration 22000: 0.013540
Cost after iteration 23000: 0.012713
Cost after iteration 24000: 0.012000
Cost after iteration 25000: 0.011377
Cost after iteration 26000: 0.01082

# Predict the results by a trained neural network.

In [0]:
predictions = network.predict(test_X, test_Y)

# Print the accuracy

In [16]:
recall = np.sum(predictions) / np.sum(test_Y)
print('Recall: %d%%' % (recall * 100))

Recall: 99%


In [17]:
precision = np.sum(test_Y == predictions) / predictions.size
print('Recall: %d%%' % (precision * 100))

Recall: 99%


In [18]:
TP = np.squeeze(test_Y.astype('uint32') @ predictions.T.astype('uint32'))
TN = np.squeeze((1 - test_Y.astype('uint32')) @ (1 - predictions.T.astype('uint32')))
accuracy = ( TP + TN ) / predictions.size
print('Accuracy: %d%%' % (accuracy * 100))

Accuracy: 99%
